# Association Rules


## Import Libraries and Load Data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules

In [ ]:
# Load data
Borrowings_Table = pd.read_excel('../data/Clean_Data/cleaned_borrowings.xlsx')

# Group by user to create transactions
Borrowings_transactions = Borrowings_Table.groupby('N° lecteur')['Titre_clean'].apply(list).reset_index()

# Create transaction list
transactions = Borrowings_transactions['Titre_clean'].tolist()

# Transform to binary matrix
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
borrowing_df = pd.DataFrame(te_ary, columns=te.columns_)

print(f'Total transactions: {len(borrowing_df)}')
print(f'Unique books: {len(borrowing_df.columns)}')
print(f'Average books per transaction: {borrowing_df.sum(axis=1).mean():.2f}')

**Categorize user by Borrowings**


In [ ]:
plt.figure(figsize=(10, 8))
Borrowings_transactions['num_books_borrowed'] = Borrowings_transactions['Titre_clean'].apply(len)
# Create borrowing categories
borrowing_categories = pd.cut(Borrowings_transactions['num_books_borrowed'],
                              bins=[0, 1, 2, 3, 5, float('inf')],
                              labels=['1 book', '2 books', '3 books', '4-5 books', '6+ books'])

category_counts = borrowing_categories.value_counts().sort_index()

# Create pie chart
colors = plt.cm.Set3(range(len(category_counts)))
wedges, texts, autotexts = plt.pie(category_counts, 
                                     labels=category_counts.index,
                                     autopct='%1.1f%%',
                                     startangle=90,
                                     colors=colors,
                                     textprops={'fontsize': 12, 'fontweight': 'bold'})

# Add counts to labels
for i, (label, count) in enumerate(zip(category_counts.index, category_counts.values)):
    texts[i].set_text(f'{label}\n({count} users)')

plt.title('User Segmentation by Borrowing Frequency', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('user_segmentation.png', dpi=300, bbox_inches='tight')
plt.show()

## Analyze Support Distribution


In [ ]:
# Calculate support for all items
item_support = borrowing_df.sum() / len(borrowing_df)
item_support_df = pd.DataFrame({
    'item': borrowing_df.columns,
    'support': item_support.values
}).sort_values('support', ascending=False)

# Statistics
print('Support Statistics:')
print(f'  Mean: {item_support.mean():.4f}')
print(f'  Median: {item_support.median():.4f}')
print(f'  Std Dev: {item_support.std():.4f}')
print(f'  Min: {item_support.min():.4f}')
print(f'  Max: {item_support.max():.4f}')
print(f'\nTop 10 most frequent items:')
print(item_support_df.head(10).to_string(index=False))

## Support Threshold Sensitivity Analysis


In [ ]:
# Test various support thresholds
support_values = [0.003, 0.0037, 0.005, 0.0075, 0.01, 0.015, 0.02, 0.025, 0.03]
support_analysis = []

print('Testing support thresholds...')
for min_sup in support_values:
    freq_itemsets = fpgrowth(borrowing_df, min_support=min_sup, use_colnames=True)
    
    singles = freq_itemsets[freq_itemsets['itemsets'].apply(lambda x: len(x) == 1)]
    pairs = freq_itemsets[freq_itemsets['itemsets'].apply(lambda x: len(x) == 2)]
    triples = freq_itemsets[freq_itemsets['itemsets'].apply(lambda x: len(x) >= 3)]
    
    support_analysis.append({
        'min_support': min_sup,
        'total_itemsets': len(freq_itemsets),
        'single_items': len(singles),
        'pairs': len(pairs),
        'triples_plus': len(triples),
        'max_size': freq_itemsets['itemsets'].apply(len).max()
    })

support_df = pd.DataFrame(support_analysis)
print('\nSupport Analysis Results:')
print(support_df.to_string(index=False))


## Select Optimal Support


Based on the prev analysis , we fount that **min_sup=0.003** gives a **lot** of of total itemsets , but for **min_sup=0.0037/0.005** its **consistent** and managable , and the other values gives a **few** total itemsets . So we choose **0.005** as min sup ( when u doubt go higher )


In [ ]:
# Select optimal support

optimal_support = 0.005

print(f'Selected optimal support: {optimal_support}')
# Generate frequent itemsets
frequent_itemsets = fpgrowth(borrowing_df, min_support=optimal_support, use_colnames=True)
print(f'\n✓ Generated {len(frequent_itemsets)} frequent itemsets')

## Confidence Threshold Analysis


In [ ]:
# Test various confidence thresholds
confidence_values = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5]
confidence_analysis = []

print('Testing confidence thresholds...')
for min_conf in confidence_values:
    try:
        rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_conf)
        
        if len(rules) > 0:
            confidence_analysis.append({
                'min_confidence': min_conf,
                'num_rules': len(rules),
                'avg_confidence': rules['confidence'].mean(),
                'avg_lift': rules['lift'].mean(),
                'strong_rules': len(rules[rules['lift'] > 1.2])
            })
        else:
            confidence_analysis.append({
                'min_confidence': min_conf,
                'num_rules': 0,
                'avg_confidence': 0,
                'avg_lift': 0,
                'strong_rules': 0
            })
    except:
        pass

confidence_df = pd.DataFrame(confidence_analysis)
print('\nConfidence Analysis Results:')
print(confidence_df.to_string(index=False))

## Generate Final Rules with Optimal Parameters


Based on the analysis , we choose the min_confdence because it **Balance between Quality and Quantity**


In [ ]:
# Select optimal confidence
optimal_confidence = 0.3

print(f'OPTIMAL PARAMETERS SELECTED:')
print(f'  Minimum Support: {optimal_support}')
print(f'  Minimum Confidence: {optimal_confidence}')

# Generate rules
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=optimal_confidence)
rules = rules.sort_values('lift', ascending=False)

print(f'\nGenerated {len(rules)} association rules')
print(f'  Average confidence: {rules["confidence"].mean():.3f}')
print(f'  Average lift: {rules["lift"].mean():.3f}')

In [ ]:
# Display top rules
print('\n' + '='*120)
print('TOP 10 ASSOCIATION RULES (by Lift)')
print('='*120)

for idx, (i, rule) in enumerate(rules.head(10).iterrows(), 1):
    antecedent = ', '.join(list(rule['antecedents']))[:60]
    consequent = ', '.join(list(rule['consequents']))[:60]
    
    print(f'\nRule {idx}:')
    print(f'  IF user borrows: {antecedent}')
    print(f'  THEN likely to borrow: {consequent}')
    print(f'  Confidence: {rule["confidence"]:.3f} | Lift: {rule["lift"]:.3f} | Support: {rule["support"]:.4f}')
    print('-'*120)

here we noticed that the support of all the rules equals to 0.0074 , so we decided to analyse the support of all the itemsets that contains more then 1 item ( which we are gonna construct the rules from )


In [ ]:
# Filter only itemsets with size >= 2
double_plus = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) >= 2)]

# Compute support statistics
support_values = double_plus['support']

print("Support Statistics for itemsets with size >= 2:")
print(f"  Count: {len(support_values)}")
print(f"  Mean: {support_values.mean():.4f}")
print(f"  Median: {support_values.median():.4f}")
print(f"  Standard Deviation: {support_values.std():.4f}")
print(f"  Minimum Support: {support_values.min():.4f}")
print(f"  Maximum Support: {support_values.max():.4f}")

print("\nTop 10 double+ by support:")
print(double_plus.sort_values('support', ascending=False).head(10).to_string(index=False))

So why those item sets with hight support , didnt apear in the best rules ??


In [ ]:
# assuming 'rules' is your association rules DataFrame
plt.figure(figsize=(10, 7))
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.0)

# scatter plot of support vs confidence, colored by lift
scatter = plt.scatter(
    rules['support'],
    rules['confidence'],
    c=rules['lift'],
    cmap='viridis',
    alpha=0.7
)
# Threshold lines
plt.axvline(x=0.005, color='red', linestyle='--', label='Min Support = 0.005')
plt.axhline(y=0.3, color='blue', linestyle='--', label='Min Confidence = 0.3')
plt.colorbar(scatter, label='Lift')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Association Rules: Support vs Confidence (colored by Lift)')
plt.grid(True)
plt.show()

## Export Results


In [ ]:
# Export rules
rules_export = rules.copy()
rules_export['antecedents'] = rules_export['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_export['consequents'] = rules_export['consequents'].apply(lambda x: ', '.join(list(x)))
rules_export.to_csv('../data/association_result/association_rules_final.csv', index=False)


## Summary Report


In [ ]:
print('='*80)
print('ASSOCIATION RULES ANALYSIS - FINAL SUMMARY')
print('='*80)
print(f'\nOPTIMAL PARAMETERS:')
print(f'  Minimum Support: {optimal_support}')
print(f'  Minimum Confidence: {optimal_confidence}')
print(f'\nRESULTS:')
print(f'  Frequent Itemsets: {len(frequent_itemsets)}')
print(f'  Association Rules: {len(rules)}')
print(f'  Average Confidence: {rules["confidence"].mean():.3f}')
print(f'  Average Lift: {rules["lift"].mean():.3f}')
print(f'  Strong Rules (Lift>1.5): {len(rules[rules["lift"]>1.5])} ({len(rules[rules["lift"]>1.5])/len(rules)*100:.1f}%)')
print(f'\nJUSTIFICATION:')
print(f'  - Support threshold selected through sensitivity analysis')
print(f'  - Confidence threshold optimized for rule quality (lift)')
print(f'  - {len(rules[rules["lift"]>1.2])/len(rules)*100:.0f}% of rules show genuine positive associations')
print(f'  - Parameters validated with multiple quality metrics')
print('='*80)